# Embedding - Atributos Latentes - Entradas categóricas

Este notebook apresenta o conceito de embedding e como usá-lo no Keras, através dos seguintes exemplos numérico:
- Rede com entrada categórica (one-hot) e camada densa
- Embedding como forma eficiente de tratar entrada categórica
- Aplicação de um camada convolução unidimensional numa sequência categórica (com embedding)

## Importação

In [1]:
%matplotlib inline
import matplotlib.pyplot as plot
from IPython import display
from __future__ import print_function

import os
import sys
import numpy as np
import numpy.random as nr

from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalMaxPooling1D
from keras.models import Model, Sequential


Using TensorFlow backend.


## Variável categórica

Uma variável categórica pode ter um valor dentro de um conjunto limitado que represente uma categoria nominal.
Alguns exemplos de variáveis categóricas:
- Grupo sanguíneo: A, B, AB or O.
- Cidade onde uma pessoa reside
- Cor de um produto: vermelho, verde, azul
- Uma palavra, dentro de um vocabulário limitado

# Rede neural com entrada categórica

Quando a rede neural possui entradas categóricas, temos a necessidade de colocá-lo na forma 
categórica utilizando a codificação *one-hot*. 
Iremos fazer um exemplo de rede neural com apenas uma camada densa e entrada com 
dados categóricos com os seguintes parâmetros:
- entrada categórica pertencente a um conjunto de 20 classes (n_classes)
- amostra é constituída de 10 amostras categóricos (n_amostras)
- cada amostra é um número (id) entre 0 e 19: [19, 10, 0, 1, 7, 5, 0, 1, 15, 2]
- camada densa com 5 neurônios (n_neuronios)

In [3]:
n_classes = 20
n_neuronios = 5
n_amostras = 10

## Diagrama da rede neural com entradas categóricas de uma camada e sem bias

<img src='../figures/Embedding_neural.png', width = 400pt></img>

### Criação da codificação categórica (one-hot) da sequência de entrada

In [5]:
data = np.array([19, 10, 0, 1, 7, 5, 0, 1, 15, 2])
data, data.shape

(array([19, 10,  0,  1,  7,  5,  0,  1, 15,  2]), (10,))

In [6]:
nb_seqs,SEQ_LEN = sequences.shape
data_oh = to_categorical(data, n_classes).astype(np.int)
data_oh, data_oh.shape

(array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 (10, 20))

## Criação do modelo da rede densa com 5 camadas

In [8]:
model_eq = Sequential()
model_eq.add(Dense(n_neuronios, 
                   input_shape=(n_classes,),
                   use_bias=False))
model_eq.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 5)                 100       
Total params: 100
Trainable params: 100
Non-trainable params: 0
_________________________________________________________________


### Criação dos pesos da rede

Como ilustração, iremos inicializar a rede com pesos de modo que possamos identificar quando cada conjunto de pesos
for utilizado para cada símbolo categórico:
- quando a categoria for $i$, os neurônios de saída devem receber os valores $[i,2i,3i,4i,5i]$.

Os pesos da rede possuem 20 linhas (uma para cada classes de entrada) por 5 colunas (atributos de cada categoria):

In [9]:
W = np.arange(n_classes).reshape(-1,1).dot(np.arange(1,n_neuronios+1).reshape(1,-1))
W, W.shape, W.dtype

(array([[ 0,  0,  0,  0,  0],
        [ 1,  2,  3,  4,  5],
        [ 2,  4,  6,  8, 10],
        [ 3,  6,  9, 12, 15],
        [ 4,  8, 12, 16, 20],
        [ 5, 10, 15, 20, 25],
        [ 6, 12, 18, 24, 30],
        [ 7, 14, 21, 28, 35],
        [ 8, 16, 24, 32, 40],
        [ 9, 18, 27, 36, 45],
        [10, 20, 30, 40, 50],
        [11, 22, 33, 44, 55],
        [12, 24, 36, 48, 60],
        [13, 26, 39, 52, 65],
        [14, 28, 42, 56, 70],
        [15, 30, 45, 60, 75],
        [16, 32, 48, 64, 80],
        [17, 34, 51, 68, 85],
        [18, 36, 54, 72, 90],
        [19, 38, 57, 76, 95]]), (20, 5), dtype('int64'))

## Predição com as 10 amostras: [19, 10, 0, 1, 7, 5, 0, 1, 15, 2]

Observe que a predição da rede com a sequência categórica acima é feita com a substituição
da categoria com os 5 atributos de cada classe.

In [10]:
model_eq.set_weights([W])
pp = model_eq.predict(data_oh).astype(np.int)
pp, pp.shape

(array([[19, 38, 57, 76, 95],
        [10, 20, 30, 40, 50],
        [ 0,  0,  0,  0,  0],
        [ 1,  2,  3,  4,  5],
        [ 7, 14, 21, 28, 35],
        [ 5, 10, 15, 20, 25],
        [ 0,  0,  0,  0,  0],
        [ 1,  2,  3,  4,  5],
        [15, 30, 45, 60, 75],
        [ 2,  4,  6,  8, 10]]), (10, 5))

# Embedding como implementação eficiente de entradas categóricas

Nesta implementação de rede neural com entrada categórica, existem dois fatores que dificultam a sua
implementação eficiente:
- necessidade de se fazer a codificação para categórico antes de colocá-lo na rede
- se o número de classes for muito alto, a implementação pode se tornar muito ineficiente. É comum
  termos centenas de milhares de classes, como é o caso de palavras dentro de um vocabulário.
  
A camada `Embedding` implementado no Keras resolve estes dois problemas de forma eficiente:
- faz a codificação categórica automaticamente e já retorna a aplicação dos pesos nos valores categóricos

Assim, a camada `Embedding` é sempre uma camada de entrada e nela é preciso especificar o número de
classes e o número de atributos de cada classe:

O diagrama a seguir mostra a aplicação do Embedding.

<img src = '../figures/Embedding_1.png',width=700pt></img>

## Criação da mesma rede, porém agora mais eficiente, com o uso do Embedding

In [19]:
model = Sequential()
model.add(Embedding(n_classes, n_neuronios, input_length=1))
model.add(Flatten())
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1, 5)              100       
_________________________________________________________________
flatten_2 (Flatten)          (None, 5)                 0         
Total params: 100
Trainable params: 100
Non-trainable params: 0
_________________________________________________________________


## Predição com mesma sequência: [19, 10, 0, 1, 7, 5, 0, 1, 15, 2]

Confirmamos aqui que a camada Embedding é equivalente à rede densa da entrada categórica feita anteriormente.

In [20]:
model.set_weights([W])
p = model.predict(data).astype(np.int)
p, p.shape

(array([[19, 38, 57, 76, 95],
        [10, 20, 30, 40, 50],
        [ 0,  0,  0,  0,  0],
        [ 1,  2,  3,  4,  5],
        [ 7, 14, 21, 28, 35],
        [ 5, 10, 15, 20, 25],
        [ 0,  0,  0,  0,  0],
        [ 1,  2,  3,  4,  5],
        [15, 30, 45, 60, 75],
        [ 2,  4,  6,  8, 10]]), (10, 5))

## Embedding como atributos latentes de um objeto categórico

Podemos interpretar o embedding como uma codificação de atributos latentes de um objeto
categórico. Por exemplo, se estamos codificando filmes, as 5 categorias visto no exemplo
acima poderiam representar a quantidade de suspense, romantismo, aventura, infantil e terror
que um filme possui. Se fosse processar palavras, os atributos poderiam representar o seu
significado (*word embedding*).

O embedding pode ser fixo (não deve ser treinado), quando sabemos exatamente os atributos
das classes ou treináveis, quando queremos que a rede utilize estes atributos como parâmetros
a serem minimizados.

# Aprendizados com este notebook